In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
import cv2
import time
from moviepy.editor import *
import matplotlib.pyplot as plt

In [2]:
class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt
        tf.disable_v2_behavior()

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.compat.v2.io.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()

In [4]:
import os
if __name__ == "__main__":
    model_path = 'frozen_inference_graph.pb'
    odapi = DetectorAPI(model_path)
    threshold = 0.7
    video_file_path = r'F:\DSGP\Violence Detection\Prediction\VioNo.mp4'
    print("file exists?", os.path.exists(video_file_path))
    output_file_path = r'C:\Users\WW\Violence Detection\violence_test_videos/Output.mp4'
    # Read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)
 
    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    # VideoWriter to store the output video in the disk.
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 
                                    video_reader.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))
 
    
    while True:
        r, img = video_reader.read()
    
        height, width , layers = img.shape
        size=(width,height)
    
        boxes, scores, classes, num = odapi.processFrame(img)

        # Visualization of the results of a detection.

        for i in range(len(boxes)):
            # Class 1 represents human
            if classes[i] == 1 and scores[i] > threshold:
                box = boxes[i]
                cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)
                
        # Write The frame into the disk using the VideoWriter
        video_writer.write(img)                       
        
    video_reader.release()
    video_writer.release()


file exists? True
Elapsed Time: 2.901738405227661
Elapsed Time: 0.1546797752380371
Elapsed Time: 0.15999484062194824
Elapsed Time: 0.1562342643737793
Elapsed Time: 0.1657731533050537
Elapsed Time: 0.16411900520324707
Elapsed Time: 0.15448689460754395
Elapsed Time: 0.17185592651367188
Elapsed Time: 0.15371489524841309
Elapsed Time: 0.17185640335083008
Elapsed Time: 0.15967774391174316
Elapsed Time: 0.1562342643737793
Elapsed Time: 0.1615290641784668
Elapsed Time: 0.1562328338623047
Elapsed Time: 0.16339731216430664
Elapsed Time: 0.1562345027923584
Elapsed Time: 0.1594550609588623
Elapsed Time: 0.17185640335083008
Elapsed Time: 0.15804171562194824
Elapsed Time: 0.17221689224243164
Elapsed Time: 0.16212892532348633
Elapsed Time: 0.1479170322418213
Elapsed Time: 0.1506342887878418
Elapsed Time: 0.1406102180480957
Elapsed Time: 0.14408349990844727
Elapsed Time: 0.1562340259552002
Elapsed Time: 0.14577293395996094
Elapsed Time: 0.15670442581176758
Elapsed Time: 0.14534211158752441
Elapsed Ti

AttributeError: 'NoneType' object has no attribute 'shape'

In [5]:
#output_file_path = r'C:\Users\WW\Violence Detection\violence_test_videos/Output.mp4'
VideoFileClip(output_file_path, audio=False, target_resolution=(300,None)).ipython_display(maxduration=1000)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
